1. Memuat Data dan Library yang Diperlukan

In [29]:
import pandas as pd
import numpy as np
from collections import Counter


In [30]:
data = pd.read_csv('../00_Data/Electronic_sales_Sep2023-Sep2024.csv')

2. Melakukan Data Wrangling

Data Type Checks

In [31]:
# data['Gender'] = data['Gender'].astype(str)
# data['Add-ons Purchased'] = data['Add-ons Purchased'].astype(str)

2.1 Handling Missing Values

Gender (0.005% missing)

In [32]:
data['Gender'] = np.where(
    data['Gender'].isna(),
    np.nan,
    data['Gender'].astype(str)
)
data = data.dropna(subset=['Gender'])

Add-ons Purchased (24.34% missing)

In [33]:
data['Add-ons Purchased'] = np.where(
    data['Add-ons Purchased'].isna(),
    np.nan,
    data['Add-ons Purchased'].astype(str)
)
data['Add-ons Purchased'] = data['Add-ons Purchased'].fillna('No Addons')

2.2 Handling High Cardinality

Purchase Date

In [34]:
# Convert to datetime and extract useful features
data['Purchase Date'] = pd.to_datetime(data['Purchase Date'])
data['Purchase_Day'] = data['Purchase Date'].dt.day
data['Purchase_Month'] = data['Purchase Date'].dt.month
data['Purchase_Year'] = data['Purchase Date'].dt.year
# data['Purchase_DayOfWeek'] = data['Purchase Date'].dt.dayofweek
# data['Purchase_DayOfYear'] = data['Purchase Date'].dt.dayofyear

# Drop original column if needed
data.drop('Purchase Date', axis=1, inplace=True)

Add-ons Purchased

In [35]:
data['Add-ons Purchased'] = data['Add-ons Purchased'].str.replace(', ', ',')

# Pisahkan string menjadi list
split_data = data['Add-ons Purchased'].str.split(',')

all_items = []
for items in split_data:
    all_items.extend(items)
unique_categories = set(all_items)

for category in unique_categories:
    data[f"{category}_count"] = split_data.apply(lambda x: x.count(category))


In [36]:
data.head()

,Customer ID,Age,Gender,Loyalty Member,Product Type,SKU,Rating,Order Status,Payment Method,Total Price,...,Shipping Type,Add-ons Purchased,Add-on Total,Purchase_Day,Purchase_Month,Purchase_Year,Accessory_count,Impulse Item_count,No Addons_count,Extended Warranty_count
0,1000,53,Male,No,Smartphone,SKU1004,2,Cancelled,Credit Card,5538.33,...,Standard,"Accessory,Accessory,Accessory",40.21,20,3,2024,3,0,0,0
1,1000,53,Male,No,Tablet,SKU1002,3,Completed,Paypal,741.09,...,Overnight,Impulse Item,26.09,20,4,2024,0,1,0,0
2,1002,41,Male,No,Laptop,SKU1005,3,Completed,Credit Card,1855.84,...,Express,No Addons,0.00,17,10,2023,0,0,1,0
3,1002,41,Male,Yes,Smartphone,SKU1004,2,Completed,Cash,3164.76,...,Overnight,"Impulse Item,Impulse Item",60.16,9,8,2024,0,2,0,0
4,1003,75,Male,Yes,Smartphone,SKU1001,5,Completed,Cash,41.50,...,Express,Accessory,35.56,21,5,2024,1,0,0,0


In [37]:
# Save Data
data.to_csv('../00_Data/electronic_sales_prepared.csv', index=False)